# Proyecto de laboratorio N°1

Grupo: 1

Estudiantes:

-Constanza Olivos Fernandez

-Javier Nanco Becerra

-Nicolás Pozo Villagrán

Fecha: 17-09-2025

# **Objetivos del notebook**

El objetivo de este notebook es desarrollar un flujo completo de análisis predictivo orientado a estimar el rendimiento del trigo (kg/ha) a partir de variables agroclimáticas, edáficas y de manejo agrícola, utilizando un dataset existente (features_trigo.csv). Este dataset fue preprocesado aplicando transformaciones como codificación de variables categóricas y normalización de las numéricas, definiendo el rendimiento como variable objetivo del estudio.

Posteriormente, se entrenaron distintos modelos de aprendizaje automático con el fin de comparar enfoques y evaluar su desempeño. Se incluyeron modelos clásicos como Regresión Lineal, Ridge, Lasso, Árboles de Decisión, Random Forest, SVR y KNN; modelos modernos como XGBoost, LightGBM y CatBoost; y finalmente un modelo de red neuronal MLP como aproximación de aprendizaje profundo. Cada modelo fue evaluado mediante métricas de regresión ampliamente utilizadas: R² para medir el poder explicativo, RMSE y MAE para cuantificar los errores en las predicciones, y MAPE para expresar el error en términos porcentuales.

Además del cálculo de métricas, se realizó un análisis de residuos, graficando los errores en función de los valores predichos, lo que permitió detectar posibles sesgos sistemáticos en las estimaciones. Estos análisis, junto con la comparación de métricas, permitieron seleccionar los modelos más prometedores, considerando no solo su precisión numérica sino también su aplicabilidad en el contexto agrícola.

En conjunto, este notebook busca comparar de manera estructurada distintos enfoques de Machine Learning y Deep Learning para determinar cuáles resultan más adecuados para la predicción del rendimiento del trigo. Los resultados obtenidos permiten, además, generar interpretaciones que vinculan los modelos con el conocimiento agronómico, aportando una herramienta de apoyo a la toma de decisiones en la gestión de cultivos.

# 3. Documentación del Proceso de Desarrollo

# AQUI VAN LOS PIP

# 2.4-. README – Features Derivadas – Dataset Sintético de Trigo (La Araucanía)

## 1. Descripción
Este conjunto de **features derivadas** fue creado a partir del dataset sintético de trigo para mejorar la predicción del rendimiento en **invierno y primavera**.  

Las variables se construyeron a partir de condiciones climáticas, nutricionales y de manejo agrícola, con el fin de capturar **patrones adicionales de crecimiento, fertilidad y estrés** que no están explícitos en las variables originales.  

---

## 2. Features derivadas

| **Nombre**            | **Tipo**    | **Descripción**                                   | **Cómo se calcula** | **Justificación / Notas** |
|------------------------|-------------|---------------------------------------------------|---------------------|---------------------------|
| **GDD_norm**           | Numérica    | Grados-día normalizados por radiación solar       | `GDD / Horas_luz`   | Relaciona calor útil con horas de luz, midiendo la eficiencia térmica de cada temporada. |
| **Indice_fertilidad**  | Numérica    | Índice ponderado de nutrientes (N, P, K)          | `0.5*N + 0.3*P + 0.2*K` | Resume la fertilización en una métrica única. Los pesos reflejan mayor importancia del Nitrógeno. |
| **Clima_reglas**       | Categórica  | Clasificación climática por reglas simples        | Si `Lluvia > 600` y `Temp_media < 18` → `húmedo_frío`; si `Lluvia < 300` y `Temp_media > 20` → `seco_cálido`; sino → `moderado` | Identifica condiciones extremas que afectan el cultivo. |
| **Rango_termico_cat**  | Categórica  | Clasificación del rango térmico                   | `<6` = `estable`; `6–10` = `moderado`; `>10` = `variable` | Permite distinguir temporadas con alta o baja amplitud térmica. |
| **Labranza_binary**    | Binaria     | Tipo de labranza codificado                       | `1` si convencional, `0` si cero labranza | Facilita el uso de labranza en modelos de ML. |
| **Plagas_binary**      | Binaria     | Presencia de plagas                               | `1` si “sí”, `0` si “no” | Convierte un factor cualitativo en cuantitativo. |
| **Deficiencia_binary** | Binaria     | Deficiencia nutricional                           | `1` si “sí”, `0` si “no` | Facilita medir impacto de deficiencias. |
| **Problema**           | Numérica    | Total de problemas agrícolas                      | `Plagas_binary + Deficiencia_binary` | Resume en un solo indicador el efecto de plagas y deficiencias. |

---

## 3. Notas generales
- Las **features derivadas** enriquecen el dataset y permiten que los modelos de machine learning detecten patrones no lineales.  
- Dado que los datos son solo de **invierno y primavera**, no se incluyeron acumulados de lluvia.  
- Se recomienda usar las variables originales **junto con estas derivadas** para el entrenamiento.  
- Las variables categóricas (`Clima_reglas`, `Rango_termico_cat`) pueden transformarse a **one-hot encoding** si el modelo lo requiere.  


In [ ]:
import pandas as pd
import numpy as np

# ========================
# Cargar CSV original
# ========================
df = pd.read_csv('Prueba2.csv', sep=',')

# ========================
# VARIABLES DERIVADAS BÁSICAS
# ========================

# 1. Grados-día de crecimiento normalizado por horas de luz
df['GDD_norm'] = df['Temp_media'].apply(lambda x: max(x - 10, 0)) / df['Horas_luz']

# 2. Fertilidad total ponderada
df['Indice_fertilidad'] = 0.5*df['N_kg_ha'] + 0.3*df['P_kg_ha'] + 0.2*df['K_kg_ha']

# 3. Categoría de clima según reglas simples
def clasificar_clima(row):
    if row['Lluvia_mm'] > 600 and row['Temp_media'] < 18:
        return 'húmedo_frío'
    elif row['Lluvia_mm'] < 300 and row['Temp_media'] > 20:
        return 'seco_calido'
    else:
        return 'moderado'
df['Clima_reglas'] = df.apply(clasificar_clima, axis=1)

# 4. Rango térmico
df['Rango_termico'] = df['Temp_max'] - df['Temp_min']

# 5. Rango térmico categorizado
def categorizar_rango(rango):
    if rango < 6:
        return 'estable'
    elif rango <= 10:
        return 'moderado'
    else:
        return 'variable'
df['Rango_termico_cat'] = df['Rango_termico'].apply(categorizar_rango)

# ========================
# VARIABLES BINARIAS
# ========================
df['Labranza_binary'] = df['Labranza'].apply(lambda x: 1 if x=='convencional' else 0)
df['Plagas_binary'] = df['Plagas'].apply(lambda x: 1 if x=='sí' else 0)
df['Deficiencia_binary'] = df['Deficiencia'].apply(lambda x: 1 if x=='sí' else 0)
df['Problema'] = df['Plagas_binary'] + df['Deficiencia_binary']

# ========================
# Guardar CSV final con todas las features derivadas
# ========================
df.to_csv('a.csv', index=False)
print("✅ CSV con features derivadas creado y listo para modelos de ML.")


# FASE 3

In [7]:
# Cargar dataset
df = pd.read_csv("Prueba2.csv")

In [15]:
import pandas as pd

# ===== Estadísticas de variables numéricas =====
numeric_columns = ['Lluvia_mm', 'Horas_luz', 'Temp_media', 'Temp_max', 'Temp_min', 
                   'N_kg_ha', 'P_kg_ha', 'K_kg_ha', 'pH', 'Rendimiento_kg_ha']
numeric_df = df[numeric_columns]

# Calcular estadísticas descriptivas
stats = numeric_df.describe(percentiles=[.25, .5, .75, .90, .95]).T
stats['varianza'] = numeric_df.var()
stats['rango'] = stats['max'] - stats['min']

# Reorganizar y renombrar columnas
stats = stats[['count', 'mean', 'std', 'varianza', 'min', '25%', '50%', '75%', '90%', '95%', 'max', 'rango']]
stats.columns = ['n', 'media', 'desv_estándar', 'varianza', 'mínimo', 'P25', 'P50', 'P75', 'P90', 'P95', 'máximo', 'rango']
formatted_stats = stats.round(2)

# ===== Mostrar tabla de estadísticas numéricas =====
line = "="*100
print(line)
print("TABLA DE ESTADÍSTICAS DESCRIPTIVAS - DATASET DE TRIGO SINTÉTICO")
print(line)
print(formatted_stats.to_string())
print(line)

# ===== Distribución de variables categóricas =====
categorical_vars = ['Temporada', 'Labranza', 'Plagas', 'Deficiencia']

print("\n" + "="*60)
print("DISTRIBUCIÓN DE VARIABLES CATEGÓRICAS")
print("="*60)

for var in categorical_vars:
    counts = df[var].value_counts()
    percentages = (df[var].value_counts(normalize=True) * 100).round(2)
    print(f"\n{var}:")
    for valor in counts.index:
        print(f"  {valor}: {counts[valor]} registros ({percentages[valor]}%)")


TABLA DE ESTADÍSTICAS DESCRIPTIVAS - DATASET DE TRIGO SINTÉTICO
                         n    media  desv_estándar   varianza   mínimo      P25      P50      P75      P90      P95   máximo    rango
Lluvia_mm          50000.0   454.71         205.10   42065.38   138.32   252.82   525.92   654.86   696.41   718.56   840.79   702.47
Horas_luz          50000.0    76.50           9.20      84.70    47.12    68.74    76.54    84.15    88.42    90.63   104.16    57.04
Temp_media         50000.0    17.29           2.76       7.61     8.39    15.27    17.28    19.32    20.91    21.75    27.02    18.64
Temp_max           50000.0    21.79           2.90       8.40    11.53    19.70    21.78    23.88    25.58    26.53    32.14    20.62
Temp_min           50000.0    12.80           2.90       8.40     2.57    10.71    12.80    14.88    16.57    17.52    22.96    20.39
N_kg_ha            50000.0   188.17          39.30    1544.23   120.00   154.12   188.31   222.22   242.68   249.28   256.00   136.0